## **USER ACTION REQUIRED**

- Upload ```silero_vad.onnx```
- Upload ```waves_snr.txt```
- Upload ```audio_array.pkl``` as necessary

In [ ]:
num_test_examples = 80

## **Define ASR Model**

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
!pip uninstall -y torch torchvision torchaudio triton nvidia-cublas-cu12 nvidia-cuda-runtime-cu12 nvidia-cudnn-cu12 nvidia-cufft-cu12 nvidia-curand-cu12 nvidia-cusolver-cu12 nvidia-cusparse-cu12 nvidia-nccl-cu12

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: triton 3.2.0
Uninstalling triton-3.2.0:
  Successfully uninstalled triton-3.2.0
Found existing installation: nvidia-cublas-cu12 12.5.3.2
Uninstalling nvidia-cublas-cu12-12.5.3.2:
  Successfully uninstalled nvidia-cublas-cu12-12.5.3.2
Found existing installation: nvidia-cuda-runtime-cu12 12.5.82
Uninstalling nvidia-cuda-runtime-cu12-12.5.82:
  Successfully uninstalled nvidia-cuda-runtime-cu12-12.5.82
Found existing installation: nvidia-cudnn-cu12 9.3.0.75
Uninstalling nvidia-cudnn-cu12-9.3.0.75:
  Successfully uninstalled nvidia-cudnn-cu12-9.3.0.75
Found exi

In [3]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu125

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu125
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install -q datasets bitsandbytes accelerate onnxruntime evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [5]:
import torch
import triton
import torch.backends.cudnn as cudnn

print("Torch Version:", torch.__version__)
print("Torch CUDA Version:", torch.version.cuda)
print("CUDA Available:", torch.cuda.is_available())
print("Triton Version:", triton.__version__)
print("cuDNN Version:", cudnn.version())

Torch Version: 2.6.0+cu124
Torch CUDA Version: 12.4
CUDA Available: True
Triton Version: 3.2.0
cuDNN Version: 90100


In [6]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from google.colab import drive
import time
import warnings
import numpy as np
import onnxruntime
from datasets import load_dataset
from itertools import islice
from torch.utils.data import IterableDataset
import json
import os
import wave
import threading
import textwrap
from IPython.display import clear_output
import evaluate
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
import collections
import pickle
import random

<br/>
<br/>
<br/>

**User Action Required**
- Define model to use

In [8]:
# Google Drive Env Setup
whisper_ver = 'whisper-base'
checkpoint_num = '5400'
drive.mount('/content/drive')
checkpoint_path = f'/content/drive/My Drive/{whisper_ver}-noiseaugmented-continued-checkpoints/checkpoint-{checkpoint_num}'

# Model setup code for fine-tuned whisper
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = WhisperForConditionalGeneration.from_pretrained(checkpoint_path).to(device)
model.config.use_cache = True
processor = WhisperProcessor.from_pretrained(f"openai/{whisper_ver}", language="en", task="transcribe")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

## **Define VAD model-related dependencies**

In [9]:
class OnnxWrapper():
    '''
    Code taken from: https://github.com/snakers4/silero-vad/blob/master/src/silero_vad/utils_vad.py

    See https://github.com/collabora/WhisperLive/blob/main/whisper_live/vad.py
    '''
    def __init__(self, path, force_onnx_cpu=False):
        opts = onnxruntime.SessionOptions()
        opts.inter_op_num_threads = 1
        opts.intra_op_num_threads = 1

        if force_onnx_cpu and 'CPUExecutionProvider' in onnxruntime.get_available_providers():
            self.session = onnxruntime.InferenceSession(path, providers=['CPUExecutionProvider'], sess_options=opts)
        else:
            self.session = onnxruntime.InferenceSession(path, sess_options=opts)

        self.reset_states()
        if '16k' in path:
            warnings.warn('This model support only 16000 sampling rate!')
            self.sample_rates = [16000]
        else:
            self.sample_rates = [8000, 16000]

    def _validate_input(self, x, sr: int):
        if x.dim() == 1:
            x = x.unsqueeze(0)
        if x.dim() > 2:
            raise ValueError(f"Too many dimensions for input audio chunk {x.dim()}")

        if sr != 16000 and (sr % 16000 == 0):
            step = sr // 16000
            x = x[:,::step]
            sr = 16000

        if sr not in self.sample_rates:
            raise ValueError(f"Supported sampling rates: {self.sample_rates} (or multiply of 16000)")
        if sr / x.shape[1] > 31.25:
            raise ValueError("Input audio chunk is too short")

        return x, sr

    def reset_states(self, batch_size=1):
        self._state = torch.zeros((2, batch_size, 128)).float()
        self._context = torch.zeros(0)
        self._last_sr = 0
        self._last_batch_size = 0

    def __call__(self, x, sr: int):

        x, sr = self._validate_input(x, sr)
        num_samples = 512 if sr == 16000 else 256

        if x.shape[-1] != num_samples:
            raise ValueError(f"Provided number of samples is {x.shape[-1]} (Supported values: 256 for 8000 sample rate, 512 for 16000)")

        batch_size = x.shape[0]
        context_size = 64 if sr == 16000 else 32

        if not self._last_batch_size:
            self.reset_states(batch_size)
        if (self._last_sr) and (self._last_sr != sr):
            self.reset_states(batch_size)
        if (self._last_batch_size) and (self._last_batch_size != batch_size):
            self.reset_states(batch_size)

        if not len(self._context):
            self._context = torch.zeros(batch_size, context_size)

        x = torch.cat([self._context, x], dim=1)
        if sr in [8000, 16000]:
            ort_inputs = {'input': x.numpy(), 'state': self._state.numpy(), 'sr': np.array(sr, dtype='int64')}
            ort_outs = self.session.run(None, ort_inputs)
            out, state = ort_outs
            self._state = torch.from_numpy(state)
        else:
            raise ValueError()

        self._context = x[..., -context_size:]
        self._last_sr = sr
        self._last_batch_size = batch_size

        out = torch.from_numpy(out)
        return out

    def audio_forward(self, x, sr: int):
        outs = []
        x, sr = self._validate_input(x, sr)
        self.reset_states()
        num_samples = 512 if sr == 16000 else 256

        if x.shape[1] % num_samples:
            pad_num = num_samples - (x.shape[1] % num_samples)
            x = torch.nn.functional.pad(x, (0, pad_num), 'constant', value=0.0)

        for i in range(0, x.shape[1], num_samples):
            wavs_batch = x[:, i:i+num_samples]
            out_chunk = self.__call__(wavs_batch, sr)
            outs.append(out_chunk)

        stacked = torch.cat(outs, dim=1)
        return stacked.cpu()




class VoiceActivityDetector:
    '''
    See https://github.com/collabora/WhisperLive/blob/main/whisper_live/vad.py

    Onxx model:
    '''
    def __init__(self, path, threshold=0.5, frame_rate=16000):
        """
        Initializes the VoiceActivityDetector with a voice activity detection model and a threshold.

        Args:
            threshold (float, optional): The probability threshold for detecting voice activity. Defaults to 0.5.
        """
        self.model = OnnxWrapper(path=path,force_onnx_cpu=True)
        self.threshold = threshold
        self.frame_rate = frame_rate

    def __call__(self, audio_frame):
        """
        Determines if the given audio frame contains speech by comparing the detected speech probability against
        the threshold.

        Args:
            audio_frame (np.ndarray): The audio frame to be analyzed for voice activity. It is expected to be a
                                      NumPy array of audio samples.

        Returns:
            bool: True if the speech probability exceeds the threshold, indicating the presence of voice activity;
                  False otherwise.
        """
        speech_probs = self.model.audio_forward(torch.from_numpy(audio_frame.copy()), self.frame_rate)[0]
        return torch.any(speech_probs > self.threshold).item()

## **Load HF Datasets**

**User Action Required**

- Define the number of test samples

In [ ]:
dataset_repo_test = "johnlohjy/imda_nsc_p3_test_noiseaugmented"
dataset_test = load_dataset(dataset_repo_test, split='test', streaming=True, trust_remote_code=True)

imda_nsc_p3_test_noiseaugmented.py:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

In [ ]:
class SlicedDataset(IterableDataset):
    def __init__(self, dataset, num_examples):
        self.dataset = dataset
        self.num_examples = num_examples

    def __iter__(self):
        return islice(iter(self.dataset), self.num_examples)

    def __len__(self):
        return self.num_examples

dataset_test_reduced = SlicedDataset(dataset_test, num_examples=num_test_examples)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## **Loop to simulate live transcription (from a HuggingFace Datasets)**

## **Define Server-Related Classes for ```HF Dataset```**

In [23]:
# Helper function Provided by Alexander Veysov
# See
# - https://github.com/snakers4/silero-vad/blob/master/examples/pyaudio-streaming/pyaudio-streaming-examples.ipynb
# Or (different implementations)
# - https://github.com/snakers4/silero-vad/blob/master/examples/microphone_and_webRTC_integration/microphone_and_webRTC_integration.py
def int2float(sound):
    abs_max = np.abs(sound).max()
    sound = sound.astype('float32')
    if abs_max > 0:
        sound *= 1/32768
    sound = sound.squeeze()  # depends on the use case
    return sound


class Client:
    def __init__(self, transcriber, transcriber_processor, RATE=16000, buffer_limit=45, finalized_seg_len=15, lagging_cap=25, prev_context_len=20, idle_time=0.512):
        self.frames_np = None # Store frames buffer as a numpy array
        self.frames_offset = 0.0 # Track frames offset from the very start (global timing). Tracks the duration of audio discarded
        self.timestamp_offset = 0.0 # Track transcription offset from the very start (global timing). Tracks the start of the transcription
        self.transcriber = transcriber # Initialize Whisper ASR model
        self.transcriber_processor = transcriber_processor # Initialize Whisper ASR processor
        self.transcript = [] # Store fully transcribed segments
        self.lock = threading.Lock() # for shared resources: frames_np. https://realpython.com/python-thread-lock/#threadinglock-for-primitive-locking
        self.RATE = RATE

        # Thread to run speech-to-text function
        '''
        Main "Entry point" for client
        '''
        self.running = True
        self.trans_thread = threading.Thread(target=self.speech_to_text)
        self.trans_thread.start()

        self.buffer_limit = buffer_limit # Max length of audio we will store in the RAM
        self.finalized_seg_len = finalized_seg_len # Length of each 'finalized' segment in seconds
        self.lagging_cap = lagging_cap # Max allowabe seconds of audio to transcribe before updating timestamp_offset -> Prevent lag because Whisper may take too long on longer audio
        self.eos = False # End-Of-Speech Flag for super long silence

        self.prev_transcription_seg = ''
        self.prev_context_len = prev_context_len
        self.idle_time = idle_time

    def add_frames(self, frame_np):
        self.lock.acquire()
        # If the buffer is more than buffer_limit (seconds)
        if self.frames_np is not None and self.frames_np.shape[0] > self.buffer_limit*self.RATE:
            # Save RAM:
            # Increase frames_offset by 30s
            # Discard oldest 30s of audio from buffer
            self.frames_offset += 30.0
            self.frames_np = self.frames_np[int(30*self.RATE):]
            # Update timestamp_offset if its less than the frames_offset
            if self.timestamp_offset < self.frames_offset:
                self.timestamp_offset = self.frames_offset

        # If the frame buffer is empty, initialise it with the new audio frames
        if self.frames_np is None:
            self.frames_np = frame_np.copy()
        # Else, append the new audio chunk to the existing buffer
        else:
            self.frames_np = np.concatenate((self.frames_np, frame_np), axis=0)
        # unlock the critical section of code above
        self.lock.release()


    def speech_to_text(self):
        while self.running:
            # Wait for some chunks to arrive
            if self.frames_np is None:
                time.sleep(0.02)
                continue

            # Adjust the timestamp_offset/transcription tracker too much audio for Whisper to transcribe (long processing time),
            # resulting in laggy transcriptions
            self.clip_audio_if_no_valid_segment()

            # Get the audio to be transcribed using the timestamp_offset/transcription tracker
            input_bytes, duration = self.get_audio_chunk_for_processing()

            # Prevent hallucination on too short audio
            if duration < 0.4:
                continue

            # Transcribe the audio
            input_sample = input_bytes.copy()
            self.transcribe_audio(input_sample, duration)


    def clip_audio_if_no_valid_segment(self):
        '''
        If there is > {lagging_cap} seconds of audio to transcribe in the buffer,
        adjust timestamp_offset such that its only 5s behind the total audio added so far

        If there is <= {lagging_cap} seconds of audio to transcribe, its okay

        self.timestamp_offset - self.frames_offset = Audio in the buffer that has already been transcribed
        '''
        with self.lock:
            # If there is more than {lagging_cap} seconds of audio to transcribe
            if self.frames_np[int((self.timestamp_offset - self.frames_offset)*self.RATE):].shape[0] > self.lagging_cap * self.RATE:
                # Adjust timestamp_offset s.t it is only 5s behind the total audio added so far
                duration = self.frames_np.shape[0] / self.RATE
                self.timestamp_offset = self.frames_offset + duration - 5


    def get_audio_chunk_for_processing(self):
        '''
        Get the audio to be transcribed from the buffer

        self.timestamp_offset - self.frames_offset = Audio in the buffer that has already been transcribed
        '''
        with self.lock:
            samples_take = max(0, (self.timestamp_offset - self.frames_offset) * self.RATE)
            input_bytes = self.frames_np[int(samples_take):].copy()
        duration = input_bytes.shape[0] / self.RATE
        return input_bytes, duration


    def transcribe_audio(self, input_bytes, duration):
        # Add prefix
        # https://github.com/huggingface/transformers/issues/22395
        # https://discuss.huggingface.co/t/adding-prompt-context-to-whisper-with-huggingface-transformers/31070/8
        # https://discuss.huggingface.co/t/what-is-possible-to-achieve-with-whisper-prompting/64414
        if len(self.prev_transcription_seg) > self.prev_context_len:
            prefix_context = ' '.join(self.prev_transcription_seg[:self.prev_context_len])
        else:
            prefix_context = ' '.join(self.prev_transcription_seg)

        prefix_context_ids = torch.tensor(self.transcriber_processor.get_prompt_ids(prefix_context), device=device)

        input_features = self.transcriber_processor.feature_extractor(input_bytes, sampling_rate=16000).input_features[0]
        input_features = torch.tensor(input_features).unsqueeze(0).to(device)
        generated_ids = self.transcriber.generate(input_features,prompt_ids=prefix_context_ids)
        last_segment = self.transcriber_processor.tokenizer.decode(generated_ids[0], skip_special_tokens=True, prompt_ids=prefix_context_ids)

        # Print the transcriptions to be printed and handle update of timestamp_offset as required
        self.handle_transcription_output(last_segment, duration)


    def handle_transcription_output(self, last_segment, duration):
        self.send_transcription_to_client({"text": last_segment})

        if self.is_segment_complete(duration):
            self.update_timestamp_offset(last_segment, duration)
            # Add the finalized segment (finalized with full context) to the global current predictions variable
            current_prediction.append(last_segment)
            # Update the prev_transcription_seg
            self.prev_transcription_seg = last_segment.split()


    def is_segment_complete(self, duration):

        # FOR TESTING PURPOSES maybe need an or condition because what if the remaining audio we are transcribing from the file does not hit the
        # specified length of each 'finalized' segment
        # OR maybe its needed in the case where there is really super long silence


        # If the duration of audio we are transcribing is more than the specified length of each 'finalized' segment, return true i.e. complete
        # or there is super long silence
        if duration >= self.finalized_seg_len or self.get_eos():
            return True
        return False


    def send_transcription_to_client(self, segments):
        '''
        Print the transcription
        '''
        text = [segments["text"]]

        '''
        For cmd line like outputs
        os.system("cls" if os.name == "nt" else "clear")
        '''
        #clear_output(wait=True) # Delays clearing the output until new output is available -> reduce flickering

        """Prints formatted transcript text."""
        wrapper = textwrap.TextWrapper(width=60)
        for line in wrapper.wrap(text="".join(text)):
            print(line)

        # Add idle time to prevent throttling
        time.sleep(self.idle_time)


    def update_timestamp_offset(self, last_segment, duration):
        # Update the timestamp_offset as this portion of the buffer has been finalized
        # (doesn't need to be transcribed anymore)
        with self.lock:
            self.timestamp_offset += duration

    def get_eos(self):
        self.lock.acquire()
        eos_val = self.eos
        self.lock.release()
        return eos_val

    def set_eos(self, eos):
        self.lock.acquire()
        self.eos = eos
        self.lock.release()

    def stop(self):
        '''
        Signals the thread to stop gracefully.
        '''
        print("Stopping transcription thread...")
        self.running = False
        if self.trans_thread.is_alive():
            self.trans_thread.join()  # Wait for the thread to finish




class ClientManager:
    '''
    Custom client manager class to handle clients
    "connected over the WebSocket server" (not necessarily)
    '''
    def __init__(self):
        self.clients = {}

    def add_client(self, websocket, client):
        '''
        Add a WebSocket server connection info (or just a key) and its associated client
        '''
        self.clients[websocket] = client

    def get_client(self, websocket):
        '''
        Retrieve a client associated with the WebSocket server connection info (or just a key) provided
        '''
        if websocket in self.clients:
            return self.clients[websocket]
        return False




class Server:
    def __init__(self, vad_detector, no_voice_activity_threshold=20):
        '''
        In charge of receiving, checking VA and passing audio to client
        '''
        self.client_manager = None
        self.vad_detector = vad_detector # VAD detector

        self.no_voice_activity_chunks = 0
        self.no_voice_activity_threshold = no_voice_activity_threshold


    def recv_audio(self, websocket, audio_array):
        """
        DEPRECATED entry point for transcription
        - kept in case of conversion to websockets
        Handle the new connection
        Continously process audio frames
        """
        # Handle the new connection
        if not self.handle_new_connection(websocket):
            return

        # Continously process audio frames
        while True:
            if not self.process_audio_frames(websocket, audio_array):
                break


    def handle_new_connection(self, websocket, transcriber, transcriber_processor, RATE=16000, buffer_limit=45, finalized_seg_len=15, lagging_cap=25, prev_context_len=20, idle_time=0.512):
        '''
        Initialise the client manager
        Initialise the new client and add it to the client manager
        '''

        # Initialise the client manager if not done
        if self.client_manager is None:
            self.client_manager = ClientManager()

        # Initialise the new client and add it to the client manager.
        # The websocket acts as the key, or just use any other key
        self.initialize_client(websocket, transcriber, transcriber_processor, RATE, buffer_limit, finalized_seg_len, lagging_cap)

        return True


    def initialize_client(self, websocket, transcriber, transcriber_processor, RATE=16000, buffer_limit=45, finalized_seg_len=15, lagging_cap=25, prev_context_len=20, idle_time=0.512):
        '''
        Initialize the new client and add it to the client manager
        '''
        # Initialize the new client
        client = Client(transcriber, transcriber_processor, RATE, buffer_limit, finalized_seg_len, lagging_cap, prev_context_len, idle_time)

        # Add the client to the client manager
        self.client_manager.add_client(websocket, client)


    def process_audio_frames(self, websocket, audio_array):
        # Get the audio chunk as a numpy array
        frame_np, audio_float32 = self.get_audio_from_websocket(websocket, audio_array)

        # Get the client using its associated key
        client = self.client_manager.get_client(websocket)

        # Check for voice activity in the audio chunk
        voice_active = self.voice_activity(websocket, audio_float32)

        # If there is voice activity, add audio chunk to the client's buffer
        if voice_active:
            client.add_frames(frame_np)

            self.no_voice_activity_chunks = 0
            client.set_eos(False)

        return True


    def get_audio_from_websocket(self, websocket, frame_data):
        '''
        Receive audio chunks and create a numpy array out of it
        '''
        # Creates numpy array without copying it (more efficient)
        # MAKE SURE DATA FORMATS ARE CORRECT
        # VAD:
        # - https://github.com/snakers4/silero-vad/blob/master/examples/pyaudio-streaming/pyaudio-streaming-examples.ipynb
        # - https://github.com/snakers4/silero-vad/blob/master/examples/microphone_and_webRTC_integration/microphone_and_webRTC_integration.py

        return ( frame_data, frame_data )


    def voice_activity(self, websocket, frame_np, no_voice_activity_threshold=20):
        # If there is no voice activity in the audio chunk return False, else return True
        if not self.vad_detector(frame_np):

            self.no_voice_activity_chunks += 1
            if self.no_voice_activity_chunks > self.no_voice_activity_threshold:
                client = self.client_manager.get_client(websocket)
                client.set_eos(True)
                time.sleep(0.1) # Sleep 100m; wait some voice activity

            return False
        return True

    def shutdown(self):
        '''
        Stops all client threads before shutting down the server.
        '''
        print("Stopping all clients")
        for websocket, client in self.client_manager.clients.items():
            client.stop()
        print("Server shutdown complete.")

In [24]:
##################SETTINGS########################
chunk = 4096 # number of samples to take per read
RATE = 16000 # number of samples taken per second
'''
chunk duration = chunk_size/sampling_rate
number of chunks in a single second of data = sampling_rate/chunk_size
4096/16000 = 0.256s
'''

finalized_seg_len=30 # Length of each 'finalized' segment in seconds: 20,15,10
# Max allowabe seconds of audio to transcribe before updating timestamp_offset -> Prevent lag because Whisper may take too long on longer audio. HAS to be more than finalized_seg_len
# I think this is the limit at which Whisper will take too long to transcribe and result in laggy transcription
lagging_cap=finalized_seg_len+10
buffer_limit=90 # Max length of audio we will store in the RAM

prev_context_len=20 # number of words of prev segment to provide some pre-context
idle_time=0.512 # idle time to prevent throttling

vad_model_path = './silero_vad.onnx'
vad_threshold = 0.5
vad_detector = VoiceActivityDetector(vad_model_path,vad_threshold,RATE) # Load the Silero VAD model
no_voice_activity_threshold=20 # 20*0.256=5.12 seconds
##################SETTINGS########################





##################INITIALIZATIONS########################
# Initialise Server
server = Server(vad_detector,no_voice_activity_threshold)
server.handle_new_connection("websocket",model,processor,RATE, buffer_limit, finalized_seg_len, lagging_cap, prev_context_len, idle_time) # manual call. Just use "websocket" as a key.

# Simulation delay
simulation_delay = chunk/RATE

# Initialize list to store all predicted transcriptions. Global variable
predictions = []
# Initialize list to store current transcriptions. Global variable
current_prediction = []
# Initialize list to store ground truths. Global variable
references = []
# Initialize total audio length (length of all audio samples)
total_audio_len = 0
# Initialize total audio length with extra silences
total_audio_len_extra_silences = 0
# Initialize list to store wav fnames to extract SNR values later
wav_fnames = []
##################INITIALIZATIONS########################





'''
######################################################################################
# IF CONTINOUS AUDIO ARRAY DNE, COMMENT OUT:
#####################CREATE CONTINOUS AUDIO ARRAY ONCE S.T ITS REPRODUCIBLE############
# Store all audio samples
all_audio_samples = []

for sample in dataset_test_reduced:
  # Get a single audio samples data and convert from float 64 to float 32
  # Append it to all_audio_samples
  audio_sample = sample["audio"]["array"].astype(np.float32)
  all_audio_samples.append(audio_sample)
  # Increment the audio length
  total_audio_len += len(audio_sample)/16000
  total_audio_len_extra_silences += len(audio_sample)/16000
  # Insert random lengths of silence (1s to 5s) at the end of the curr sample
  silence = np.zeros(random.randint(1, 5)*RATE, dtype=np.float32)
  all_audio_samples.append(silence)
  # Increment the total_audio_len_extra_silences with silence
  total_audio_len_extra_silences += len(silence)/16000
  # Add the reference
  references.append(sample["sentence"])
  # Add the wav file name
  wav_fnames.append(os.path.splitext(sample["path"].split('/')[-1])[0])

# Concatenate all audio samples (with silences)
audio_samples = np.concatenate(all_audio_samples)

# Add no_voice_activity_threshold chunks of silence to the concatenated audio samples so the lastest segment of audio will be saved
silence = np.zeros(chunk*(no_voice_activity_threshold+5), dtype=np.float32)
audio_samples = np.concatenate((audio_samples, silence))
total_audio_len_extra_silences += len(silence)/16000
#####################CREATE CONTINOUS AUDIO ARRAY ONCE S.T ITS REPRODUCIBLE############


######################SAVE CONTINOUS AUDIO ARRAY######################
with open(f"concatenated_audio_{total_audio_len/60}_mins.pkl", "wb") as f:
    pickle.dump(audio_samples, f)
######################SAVE CONTINOUS AUDIO ARRAY######################

######################################################################################
'''


# IF CONTINOUS AUDIO ARRAY EXISTS, USE. IF NOT, COMMENT OUT:
with open("./concatenated_audio_32.892884375_mins.pkl", "rb") as f:
    audio_samples = pickle.load(f)



######################LOOP THROUGH THE CONTINOUS AUDIO ARRAY######################
# Get the number of samples of the concatenated/continous audio
num_audio_samples = len(audio_samples)

# Read {chunk} samples at a time
samples_read = 0

while samples_read < num_audio_samples:

  if samples_read + chunk > num_audio_samples:
    data = audio_samples[samples_read:]
    # FOR TESTING ONLY
    # if the remaining audio array is < 512 samples (required by SileroVAD)
    if len(data) < 512:
      data = np.concatenate((data, np.zeros(512-len(data), dtype=np.float32)))
  else:
    data = audio_samples[samples_read:samples_read+chunk]

  # Update the number of samples read
  samples_read += len(data)
  # Simulate the time the chunk takes to be 'spoken'
  time.sleep(simulation_delay)
  # Directly call process_audio_frames with the new audio chunk in bytes
  server.process_audio_frames("websocket", data)
######################LOOP THROUGH THE CONTINOUS AUDIO ARRAY######################



# Join the predictions
predictions = ' '.join(current_prediction).strip()
# Join the references
#references = ' '.join(references).strip()
# Shutdown the server
server.shutdown()

 tekam tekam how do you choose four day or total numbers mm
okay what most busybody thing i have done i i don't really
like it's it's like small events like like it's what most
busybody thing that was so weird there was this guy who just
came out of nowhere everywhere in the cemetery you know he
told us that his name was yan wei so


KeyboardInterrupt: 

**Calculate WER**

In [ ]:
metric = evaluate.load("wer")
normalizer = BasicTextNormalizer()

In [ ]:
predictions_normalized = normalizer(predictions)
references_normalized = normalizer(references)

In [ ]:
predictions_normalized

'you know ya she was from france she shared to me about her culture so ya she would she want to learn and she would when i would visit lah when i travel the house looking forward to that it s only not interesting generation gap what s the biggest similarity or difference between your generation and your parent s generation okay would you say it s more positive or negative neutral lah good for the country in the in having as in globalization is good for the country okay never ya okay tekam tekam how do you choose four day or total numbers mm okay what most busybody thing that i ve done i i don t really know like it s it s like small events like like what s most busybody thing that s all there was this guy who just came out of nowhere and we were in the cemetery you know he told us that his name was yan wei so i thought it was a little bit weird yan wei so i kept i kept repeating the name to myself yan wei and i found it really familiar when i got back home it was a very particular was a

In [ ]:
references_normalized

''

In [ ]:
references_normalized = """you know yeah she was from france so that s she shared with me about her culture lah so you know err she would show me around when she would when i would visit lah when i travel to france looking forward to that it s really not interesting generation gap what s the biggest similarity or difference between your generation and your parent s generation okay would you say it s more positive or negative neutral lah good for the country in the in having as in globalization is good for the country okay okay um never yeah okay next tikam tikam how do you choose four d or toto numbers um okay what most busybody thing i have ever done i i don t really know like it s it s like small events like like what most busybody thing is so weird there was this guy who just came up of no where when we were in the cemetery you know he told us that his name was yan wei so i thought it was little bit weird yan wei so i kept i kept repeating the name to myself yan wei and i found it really familiar when i got back home it was a very parti it was a very you know weird event okay i ve never gone through it but it s actually losing a loved one so you know i can only imagine how that would be like so yeah that s my um greatest fear in a way ya so i m describe the place i call home in singapore ever changing not e scooters um very different from you know when i was a kid um what s the question again what s what is it again describe i would just say ever changing singapore should become the next hub for financial hub the green movement being an example for the rest of the world in the sense of environmentally friendly but i think it s already moving to the direction it s good i mean ya what can be improved cars maybe making a bigger switch from your normal oh yeah definite okay yeah think is just some yes look at yeah shanghai and beijing yeah and here s the thing you know it make me think of this company call yeah tense tencent so tencent runs this application in china that ru yes exactly and you know everything that they do about their payments about what yes yes i would say probably who would you like to answer i don t know you seem like you wanted to answer it okay oh sorry what s the which singaporean sun yanzi i love her songs and i love how she is really humble and you know live a normal life even though of her global success what s that oh taxi drivers they are really interesting people you know they like to they like to talk about everything most interesting is about this guy who would talk about all his gambling yes and he will talk about his gambling experiences in marina bay sands so you know it s really it was really funny the guy such character and he would just hmm okay right beside you okay yes exactly yeah to chat with her yeah ya exactly you know on this point i had an experience in the past where i was with the force police force and this lady would always call up to the police station and talk about talk about talk about all sorts of things so then she would complain about this and complain about that give me one point thanks alright t f r what would you rather have kids or pets why wow you getting all serious with me okay so wow you re getting all serious with me best subject what s that called social studies worst subject math i suck at numbers i suck at details um yeah so you know what what s your take on on on on on success you know the say that love makes the world go round or love is what drives the world i believe in that we will bring love to the world russell peters ya don t we all go through that period in our primary or secondary school when we read russell peters yes that they are have always been experiencing they don t move out of that circle or that space so what happens is that they do not see the all the other areas which you know could like which which is there you know so people who place safety and security as the top priority are usually the people who may not be as fulfilling i d say ya so basically i just spend my time there lah playing around with all the other kids from the other the other stores yeah thus good good times kaypoh so what s the most busybody thing you ve ever done maybe you can share oh really i i also had a similar experience yeah the other time i think was in geylang i was walking down the road and basically i heard i heard this these boys this boys right i was just eating like frog porridge with a friend and there was this banana tree that s she actually just set me down talk me through things and actually forgave me and i m in that s a definitely with the consent of the other student lah so that itself actually made me learn to think more about my own actions primary six yeah yeah of course i felt different left a great great impact on me lah of course i love her so much she is like the best teacher you can never find you know what what about you do you have any yeah of course time to time i still visit her bring her some stuff missus jaiya yeah hmm yes precisely that s exactly what i learn as well yeah bonding through struggles and also leading with example yes precisely i definitely to the best of my ability yeah that s hmm makes you feel good ah yeah okay king of fruits what s the most number of durians you have ever eaten at one go do you even eat durians the seeds okay let s do the whole okay let s do seeds then thirty probably ten i try to eat lesser lah definitely i want i want to stay in shape one ya families to where they think they deserve to be at that means of a higher level lah bring them a better better life yeah so and so forth okay so if you could send a message to a specific group of singaporean who would you address and what would you say yes gambling habits you know i come to realize that a lot of taxi drivers right they are just i mean right now i would say it s more like grab what what they they normally just do this for fun they did realize they are there are a lot of people who don t have much to do and so they re just trying to find my neighbor next door is this she s a wonderful lady she s in her seventies contributed to society she s to be a teacher they re really good lah and in good in good people yeah and well thing about you think about her is i would say for neighbors always be yeah harmony lah in their work and life that way yeah be more i would i would think there d be more value lah to that than to just be earning okay yeah okay okay yeah i get you that s what that s what people say but ya and i don t don t really believe in it lah i mean do you ya okay so when was the last time you actually interacted with a foreigner oh what do you actually talked about fulfilment fulfilment ya i mean definitely if i can fulfilment is when you can achieve something larger than yourself like what i told you earlier on basically success that s fulfilment that s interesting ya i ll give you a one point dressing ya i mean i like i like how they use to dress it to me comfort is more important and what what what they dress in is definitely more comfortable as compared to what we are wearing right now don t you agree yeah of course economy yeah we use to be okay um may none happy state of mind yes i get where you are coming from so in a way they are seeking for like physical stimulation to actually compensate for what ever they can t get mentally you re right okay maybe we can go on to the next question now so why was six years so who s the better cook your mom or your grandmother grandma what what sort of dishes uh huh hello hello hello hello hello hello oh oh oh this that s very soft hello hello hello hello yup yup okay nice nice nice nice nice this okay nice nice it s like they are really working very hard and it s a bit crazy they study harder than any of us can ever study that s why like everybody gets into university not the best because it s ultimately still a bell curve competition right but um uh but the problem is right as i was talking bout my parents uh back then actually even now lah when you get a degree in china but after he came here erm he was working lah he worked for awhile my mum was having me in china err she still had me my dad came here he did work he was part of the construction firm also then err he got out construction firm then he realised he s err his degree was invalid then err did some business related thing mum had me brought err brought me over to singapore much earlier you were just seeing one and you say eh this is not too bad and you re just going for it i think that s a bit too premature as much as as good as much as good as the job sounds and maybe this job is the best sounding one out of everything but ultimately they haven t told you to sign the contract yet there is still time to think about it uh because if you earn based on commission you can earn quite a bit if you work very hard and one thing that err and one thing and one thing that s great about him right and one thing that s great about him is that he works super hard he always work super he always super hard uh ya lah for the family for himself and he he doesn t really know how to spend also he works saves up a lot of money um think that s a really good point about him why did i even talk about this in the first place yeah he was cause we we we we got like a second hand hdb and it was quite expensive also so we ll pay off our loans lah then car also car loans and stuff like that and uh i think coe prices was not very high at that point of time it s it s insane now but imagine back when i was in primary school it s it s really quite it s it s and that s where he end up now lah he s still in sales because he he s the type of guy who who is quite averse to change i don t blame him old people are generally averse to change yup and his personality also kind of averse to change you can t teach a old dog new tricks ah that s the issue but it s the same same kind of product ya that s why that that s why i took the scholarship i cannot i don t want as in like he he s able to break even but not significantly more than even he s still earning some but not not significant amounts for the loans that our family has ma but my mum is doing quite well as not bad lah ya and uh but the problem is that they didn t really buy insurance and stuff like that at a young age cause my dad lah my dad is he s he he doesn t really do a lot of planning you re right he doesn t do a lot of planning so like up to you lor i guess but are you still planning to send out more send out more err resumes so exactly as i said lah where is where is the company ah and what does the company do i still don t understand okay oh where is it that s not too bad but won t it oh sophisticated words you mean not not intricate intricate means detailed uh you you can you even describe words as intricate okay that s a that s a better description no sorry your your your voice is not oh your voice sounds so singlish so singaporean so singaporean so singaporean ya because you re you re trying to imitate someone who is not singaporean because you re trying to imitate someone who is not singaporean that s why it sounds weird because you re singaporean because you sound very singaporean save money get your get your priorities settled first is it okay lor then all all the more i can spend my money happily doing err going to expensive place for hikes if you tell me s that your job is pa my my impression is that you re just gonna do a lot of sai kang honestly speaking like oh uh add this to my calendar like err get this memo written up send to this person or err can you take the minutes for this meeting send to everyone like that doesn t sound like a job to me that s why get then then know your job better you are you are too you are too okay with things already there are terrible mentors mine s terrible right and there are good mentors good mentors are generally very very responsive very very give a lot of good directions people with good mentors already finish the project long time ago already you know right they already started studying already my my mentor is the worse i m not kidding you is four of us what one or two a bit only oh well okay i m honestly kinda excited to start studying also it s like it s like you lor as in ah i must go to work i m so tired oh i shouldn t have taken the the second shift then i ll be like i can t wait to do work again oh ridiculous lah you that s why i every time you you say yes i ll be like you sure you want or not then you say number five easy to share with other people your things there s a lot of ways in which cloud storage is better no the okay you can the only problem with cloud storage is that moving too big a file that is too big of a s err too big a size is uh depends on the speed of your internet connection and your computer err uh we should have told them when he came in lah honestly you you don t as as in we can tell them that we can tell them that like it was supposed to be but i don t mind staying a bit longer just get this over and done with why you so lazy i am a generator of conversation i keep on generating conversation anyway ya cloud storage is in fact better you re paying thirty dollars a year what if it comes then then after that you start regretting then then then then that will be quite sad and also you you are tied at least to the probation period it s for the company lah aww that s not too bad um that that s a good thing about that i guess but the the downside is that during that period of time you won t be sending any resumes lah well true true i ll but the only issue is that with that right is that uh no lah in fact it s more secure up on the whoops sorry sorry up on the on the cloud storage than it is on the hard disk of course you can lose and damage your hard disk number two your hard disk can may have corrupted files it s really cool it s they really have a whole lot of things in place if you it really depends on you lah do you have any files that you wanna save don t don t tell me snaps do you have any important files you can technically still keep on using google drive lah and if you actually really like google drive that much i guess can consider getting a cloud storage lor let me google cloud storage err actually i think dropbox is better than google drive which one do you think like more why ya but the good thing about google drive is that it s much easier to concurrently err edit documents err dropbox is uh the most basic the the one level above basic for individuals is nine ninety a month tom and paul s cafe peach juice wah i m really feeling quite tired this quite cute ah i thought you said this was ntu s ntu s stuff oh to email everyone oh okay that s that s okay interesting but do you know do you know what this project is about huh that s odd i think you missed out one huge part of it right that says please kindly turn off your cell phone or at least silent mode so it don t vibrate alright then you you you come up with the conversation topic come we talk about it i won t send you this thing then save image wah damn cute sia this one it s a woman what woman okay true uh it s so hard to focus you you come up come up with a topic i send to you first kathmandu kathmandu is damn cool it s like a mixture of indian and indian and chinese culture a lot of monks there also but then kathmandu is very india like to to a point where it s very very like a lot of people say that it s very crowded very and i don t wanna watch on the computer because if i watch on the computer it s means that is even more reason for me to pro procrastinate aww man what a dilemma right you can be my personal assistant you can help me to down download every single episode i m hungry i m not hungry my stomach is moving ya of course have you never been to serangoon nex it s like it s like it s like buying okay imagine right imagine you re buying investment you want to know about your investment right cause you don t want to it is same thing something similar you re putting an investment your investment is the job that you re getting this is not a part time job my friend this a full time job in which you want to gain experience even if you don t want to stay there full time you want to gain experience from it this not any part time job where you just wanna earn the money and zao is it at the place that i was talking about it s pretty good stuff eh no go to the one that i recommend it s better go early no we didn t go early we go right at the time we wanted to go which is at around five plus so you wanna go err jb we can go i think in two two months two and a half months time i ll be relatively freer cause of err fam family medicine posting cause you ll be working is it i i ve tried asking before i i ve tried asking before in the places i have asked which is not that many i i i give it i give it to you but the places i ve asked they don t have it what wait no not the current one oh it s okay what oh okay because different people cook it it s quite inconsistent yeah i i always see different people cooking it i ve never i would never really go out to eat ban mian ban mian is but more eh you haven t bought your thing right the the hard disk you buy later lah you want buy later you planning to get popular right i got popular membership i told you before what you never pay attention you always forget you don t care i never forget no i always i always remember stuff please please i very excited to get the watch the watch i so excited i am getting it it s coming on friday slash saturday i was telling you you want buy or not i sell to you first i will sell eighty percent of the price i bought it at not even the full price eighty dollars anyway then i was like okay he definitely must have done something for mine right so i went to scrutinise every single thing okay and then i have a habit right when i when i write documents err especially when there s so many edits going on i need to have different versions of document like one point one one point two one point three and every time i have a new version i will i will imagine you hurt yourself and then err so now there s a gap between in skin right for example from here become like that in terms of cells ah cells microscopic level becomes like that right okay you get a paper cut it takes a chunk of your flesh off then it becomes like that right then it starts to heal it what how it heals is that it heals some amount by growing more cells but the majority of which is actually acellular no cells it s actually made up of connective tissue nope connective tissue is like err your hair is connective tissue to some extent not not not really err no hair is not made of cells it s actually made up of dead cells you re right but the dead cells die and the connective tissue form over it kind of thing aiya okay it s complicated anyway it doesn t matter so like it it s it s filled connective tissue right and this connective tissue sometimes overgrow so what happens is that as cells are growing a connective tissue is being deposition so from a purely cellular thing becomes a mixture of cellular cells and connective tissue and then like uh usually right what happens is that there is a control of growth where upon reaching and through the through the subscription through the service managed to get in touch with the school in uh peru so i was like err i was asking i was talking about her i was like eh i really couldn t find a a lot of reviews on err glasgow glasgow medical university alright then uh i really wanted to find reviews lah because wait so your title is the so what officer okay but then pa and office executives are very different things you know i just hope they don t overwork you lah cause that sounds that sounds pretty tough err err entry level job then i i was so ready to just dropped it you know but then he texted back he was like err no no he he say in the group he s gonna dao me for a few days let me put to this way well he reply me just now sure i ll pass you the details later okay that s what he reply me but let me give you a bit more details on him as a person it s not it s not weird he s a bit childish and he s a bit err it s like it s like when we we discussing medical stuff right but you know like you know when you get dogs right no no when you get but buy when you buy dogs big dogs are usually calm lah chilled uh more calm more chilled small dogs they compensate by roar very aggressive ah he s freaking aggressive he s like the smallest dog i can get a freaking aggressive will punch me one i quite ch i quite cool one lah but like okay whatever you know let s be friends but always he ll always say that oh ri one wah i forgot i forgot it is the right way to i don t know whether this is the right as in i forgot the no i i forgot the right term for this already but if i m not wrong is the squeaky wheel makes the most noise right i ll only complain to you about the the extremes right so this is not a representation on med kids do you know other med kids apart from me right that s not enough for friendship as in like i i put it to you this way it s about interplay right it s about err ya that s why i say i don t have problem with you what right i never even say i got problem with you i told you right she oh no no err err as in no i try consistently but she was always forgetting that i even existed i am someone who just happily spend time alone in the room don t jio me for dinner sure i don t care but when i say something is wrong something is wrong lah i feel it s very easy for you to make generalized uh opinions and say that like oh you guys are overreacting right cause you are not in the position and is not just me apparently what i heard right is that there is some guy call ping chong he also felt the same way towards jolene and the fact that he s actually supposed to be even closer to jolene than i am because they they like they are the same clique ah stuff like that but he he honestly he told me he felt quite sad i liked her uh she just is never present mentally this not just me and i really feel that the it s always thinking bout when we are making generalisation we are always making generalisations about the people that have piqued your interest wah another one and half hours eh one hour shag shag shag whatever lah whatever lah do you think he will transcribe the conversation that is true it must be quali quantitative right there s no singaporean i come from a well educated background i come from a well educated background no lah just kidding ya it s quite a flat it s quite a flat err structure the other company ya their their company structure is quite flat but i think cause from what the ceo told me right the company culture is quite is quite nice like like how to say ya ya ya i know lah i understand but then like the impression that he gave me you didn t use the shampoo or you didn t use the spray ya oh damn ya but then but then there won t be a lot of opportunities for you to use expedia also what right but sometimes expedia might not be the cheapest ya uh huh ya correct correct okay i haven t check the shopback yet though i think can check how you know yay i give you why is is for in is is is is a what you call that for is for the for your lost"""

In [ ]:
wer = 100 * metric.compute(predictions=[predictions_normalized], references=[references_normalized])
print(f'The WER is {wer}%')

The WER is 33.21598674673846%


In [ ]:
print(f'The total audio length (just the samples) is {total_audio_len/(60)} minutes')

The total audio length (just the samples) is 0.0 minutes


In [ ]:
print(f'The total audio length (with the added silences) is {total_audio_len_extra_silences/(60)} minutes')

The total audio length (with the added silences) is 0.0 minutes


In [ ]:
snr_wavfile_values = {}

with open("waves_snr.txt", "r") as file:
    for line in file:
        key, value = line.strip().split(' ')
        snr_wavfile_values[key] = value

In [ ]:
snr_wavfile_counts = collections.defaultdict(int)
for fname in wav_fnames:
    snr_wavfile_counts[snr_wavfile_values[fname]] += 1

In [ ]:
print('The distribution of noise levels is: ')
print(snr_wavfile_counts)

The distribution of noise levels is: 
defaultdict(<class 'int'>, {'5': 20, '15': 24, '10': 15, 'none': 21})
